<a href="https://colab.research.google.com/github/Soupy1184/BigData-Assignment2/blob/master/COMP4311_Assignment2_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import pandas as pd
import numpy as np

Question #1: CLUSTERING

In [153]:
k = int(input("Enter value of k: "))
path = input("Enter path to data file")

Enter value of k: 3
Enter path to data file/content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/data.csv


In [154]:
data = pd.read_csv(path)

In [155]:
import math

def euclidDis(a, b):
  sum = 0
  for x, y in zip(a, b):
      sum += (x - y)**2
      
  return math.sqrt(sum)

In [156]:
from pandas import DataFrame

def kMeansClustering(df, k):
  currentClusters = []
  for round in range(0, 100):
    print("Round: ", round)
    # each round resets centroids to empty array
    centroids = []
    #the first round initializes centroids to 3 random samples
    if round == 0:
      centroids = df.sample(n = k)
    else:
      # for k number of centroids
      for cent in range(0, k):
        currentCentroid = df.loc[(df['Cluster_Id'] == (cent+1))]
        #create new centroids
        # gets the mean of each column and creates a new centroid
        meanValues = []
        for column in range(0, len(data.columns)-1):
          meanValues.append(currentCentroid.iloc[:,column].mean())
        meanValues.append(cent+1)
        centroids.append(meanValues)
      
      centroids = DataFrame(centroids,columns=[list(df.columns)])
    
    #calculate distance
    #for each instance 
    cluster_id = []
    for x in range(0, len(df)):
      distance = []
      row = df.iloc[x, 0:(len(df.columns)-1)].values
      #for each centroid check which has min distance
      for i in range(0, len(centroids)):
        centroid = centroids.iloc[i, 0:(len(df.columns)-1)].values
        #for each test instance, calculate distance from each centroid
        #assign cluster ID to min distance
        distance.append(euclidDis(row, centroid))

      #assign cluster id
      cluster_id.append(distance.index(min(distance)) + 1)
   
    
    #assign cluster array to dataset
    df['Cluster_Id'] = DataFrame(cluster_id)

    if np.array_equal(cluster_id, currentClusters):
      return df
    else:
      currentClusters = cluster_id    

In [157]:
clustered_data = kMeansClustering(data, k)

clustered_data.head()

Round:  0
Round:  1
Round:  2
Round:  3
Round:  4
Round:  5
Round:  6
Round:  7
Round:  8
Round:  9
Round:  10
Round:  11
Round:  12
Round:  13
Round:  14
Round:  15
Round:  16


,X1,X2,X3,X4,Cluster_Id
0,-0.59,-0.76,-1.80,-1.05,2
1,0.59,0.76,1.80,1.05,1
2,0.70,0.49,1.03,0.74,1
3,-0.70,-0.49,-1.03,-0.74,2
4,-0.11,0.96,0.65,0.50,1


In [426]:
clustered_data.to_csv('/content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/clustered_data.csv')

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

Question #2: NAIVE BAYES

In [ ]:
trainPath = input("Enter path to train data file")
testPath = input("Enter path to test data file")

# /content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/Train.csv
# /content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/Test.csv

Enter path to train data file/content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/Train.csv
Enter path to test data file/content/drive/MyDrive/Colab Notebooks/COMP-4311/Assignment2/Test.csv


In [ ]:
train = pd.read_csv(trainPath)
test = pd.read_csv(testPath)

In [ ]:
#resource: https://www.geeksforgeeks.org/python-multiply-numbers-list-3-different-ways/
#multiplies the elements of a list together

def multiplyList(myList) :
     
    # Multiply elements one by one
    result = 1
    for x in myList:
         result = result * x
    return result

In [ ]:
def naiveBayes(dfTrain, dfTest):
  # create column names list
  trainNames = list(dfTrain.columns)
  testNames = list(dfTest.columns)

  # For prior, your program will find the number of unique values in the last column - that's the number of classes.
  classes = dfTrain[trainNames[len(dfTrain.columns)-1]].unique()

  # For each class estimate and store priors. 
  priorProbability = []
  for x in range(0, len(classes)):
    priorProbability.append(len(dfTrain.loc[(dfTrain[trainNames[len(dfTrain.columns)-1]] == classes[x])]) / len(dfTrain))
    
  priors = DataFrame(priorProbability,index=classes, columns=['Prior'])

  # For likelihoods, it will do the same - for each feature, find unique values and 
  # for each unique value find the frequency for each class and then estimate likelihood. 
  # likelihood = P(B|A)
  # for each feature
  likelihoods = []
  for i in range(0, len(trainNames)-1):
    # for each unique value in the column
    uniqueValues = dfTrain[trainNames[i]].unique() 
    for j in range (0, len(dfTrain[trainNames[i]].unique())):
      probability = []
      total = len(dfTrain.loc[(dfTrain[trainNames[i]] == uniqueValues[j])])
      probability.append(uniqueValues[j])
      for x in range(0, len(classes)):
        probability.append(len(dfTrain.loc[(dfTrain[trainNames[i]] == uniqueValues[j]) & (dfTrain[trainNames[len(dfTrain.columns)-1]] == classes[x])]) / total)
      likelihoods.append(probability)

  #list of column names
  col = []
  col.append('UniqueValue')
  for x in classes:
    col.append(x)

  #create table of likelihoods
  table = DataFrame(likelihoods, columns=col)
  
  # For each test instance, you find the matching features and 
  # class and get the prior and likelihood to calculate posterior.
  right = 0
  wrong = 0

  for y in range(0, len(dfTest)):
    row = dfTest.loc[y]
    for c in classes:
      result = []
      multiply = []
      multiply.append(priors.loc[c, 'Prior'])
      for x in range(0, len(trainNames)-1):
        value = table.loc[(row[trainNames[x]] == table['UniqueValue'])]
        multiply.append(value.iloc[0][c])
      result.append(multiplyList(multiply))

    index = result.index(max(result))
    predClass = classes[index]

    if predClass == row.loc[testNames[len(testNames) - 1]]:
      #true positive or true negative
      right += 1
    else:
      #false positive or false negative
      wrong += 1

  total = right+wrong
  accuracy = right / total
  #algorithm here
  print("The accuracy is: ", accuracy)
  return

In [ ]:
naiveBayes(train, test)

The accuracy is:  0.5
